# Geostrophic adjustment and inertia-gravity waves
### MAQ - 32806, Chiel van Heerwaarden and Imme Benedict, 2016
In this tutorial you will use the previously derived shallow water equations to study the concept of geostrophic adjustment (again pioneered by Carl-Gustav Rossby). This you will do with a conceptual model that will help you understand the formation of a low pressure system out of an atmospheric disturbance.
___
### Loading the required packages
First, we load the required packages and set the physical constants.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import animation
from IPython.display import HTML
figsize_s = 6,4
figsize_l = 9,4

In [ ]:
Omega = 7.29e-5 # Angular velocity of the earth.
g = 9.81        # Gravity constant.
a = 6.371e6     # Earth radius.

___
### 1. The shallow water equations for small perturbations
In this assignment, you are going to work with the linearized shallow water equations. This means that we assume that there is no background velocity, and that the velocities and height perturbations are small. Furthermore, to simplify the dynamics, we assume the Coriolis parameter $f$ to be a constant $f_0$.

$$
\begin{eqnarray}
\nonumber
\dfrac{\partial u^\prime}{\partial t} & = & f_0 v^\prime - g\dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
\dfrac{\partial v^\prime}{\partial t} & = & -f_0 u^\prime - g\dfrac{\partial h^\prime}{\partial y}
\\
\nonumber
\dfrac{\partial h^\prime}{\partial t} & = &
- H \left( \dfrac{\partial u^\prime}{\partial x} +\dfrac{\partial v^\prime}{\partial y} \right)
\end{eqnarray}
$$

1. Explain the governing sets of equations. Why can we neglect the advection terms if the perturbations are small? And why is $h$ replaced by $H$ in the right-hand side of the third equation?
___

### 2. The shallow water equations for our experiment
In our experiment, you are going to study the behavior of a two-dimensional system. We study an initially non-moving water layer in which we introduce a dip in the height that only varies with $x$. The governing equations are

$$
\begin{eqnarray}
\nonumber
\dfrac{\partial u^\prime}{\partial t} & = & f_0 v^\prime - g \dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
\dfrac{\partial v^\prime}{\partial t} & = & -f_0 u^\prime
\\
\nonumber
\dfrac{\partial h^\prime}{\partial t} & = &
- H \dfrac{\partial u^\prime}{\partial x}
\end{eqnarray}
$$

1. Explain why the governing equations can be written as above if the initial perturbation has no gradients in $y$.
___

### 3. Setting up the experiment
Below, you can specify the setup of the experiment. The `f_multiplier` is initially set to `1` and can be adjusted later to make the earth spin faster or slower.

In [ ]:
f_multiplier = 1.
lat = 45.
f0 = f_multiplier * 2.*Omega*np.sin(np.pi/180.*lat)
L = a * (2.*np.pi) * np.cos(np.pi/180.*lat)
L_dist = 1.e6
H = 200.
h_prime = 0.1

With the conditions above, we can create our domain (ranging from `-L/2` to `L/2`). The initial condition of $u^{\prime}$, $v^{\prime}$ and $h^{\prime}$ are stored in variables `u0`, `v0` and `h0`.

In [ ]:
nx = 1501
x = np.linspace(-L/2., L/2., nx)
dx = x[1]-x[0]
u0 = np.zeros(nx)
v0 = np.zeros(nx)

# Gaussian
x_sigma = L_dist/6.
h0 = np.exp(-x**2/(2.*x_sigma**2))
h0 = - h_prime*h0/np.max(h0)

In [ ]:
pl.figure(figsize=figsize_s)
pl.plot(x, h0, 'b-')
pl.xlabel('x (m)')
pl.ylabel('h\' (m)')
pl.tight_layout()

1. Plot the height profile of $h^{\prime}$ as a function of $x$.
2. Calculate the initial potential vorticity $q$ as a variable `q0` and plot it as a function of $x$.
3. Calculate the Rossby radius of deformation $L_r \equiv \dfrac{\sqrt{g H}}{f_0}$ for this experiment.
4. Replot $q$ and $h^{\prime}$ with the horizontal axis normalized by the Rossby radius $L_r$. How large is the perturbation compared to the Rossby radius?

___
### Running the experiment
Now, you will run the model for `nt` timesteps with time increments of `dt` seconds. The output is saved every `ni` steps. The final result of $u^{\prime}$, $v^{\prime}$ and $h^{\prime}$ are stored in variables `u`, `v` and `h`. Below you find an animation that shows the time evolution that can aid you in interpreting what is happening.

In [ ]:
nt = 50000
ni = 500
dt = 10.

In [ ]:
# MODEL INTEGRATOR, THIS MIGHT TAKE A WHILE!
u = np.copy(u0)
v = np.copy(v0)
h = np.copy(h0)

h_out = np.zeros((nt//ni+1, nx))
h_out[0,:] = h0

for n in range(1,nt+1):
    u += dt * (   f0*v - g*np.gradient(h,dx) )
    v += dt * ( - f0*u )
    h += dt * ( - H*np.gradient(u,dx) )
    
    if (n%ni == 0):
        h_out[n//ni,:] = h[:]

In [ ]:
# ANIMATION CREATION PHASE 1. PLEASE IGNORE THIS FIGURE, WE DO NOT KNOW HOW TO SUPPRESS.
# Set up the figure, the axis, and the plot element we want to animate
fig, ax = pl.subplots(figsize=figsize_l)
xmin, xmax = min(x), max(x)
hmin, hmax = np.min(h_out), np.max(h_out)
hmin -= 0.05*(hmax-hmin)
hmax += 0.05*(hmax-hmin)
ax.set_xlim(xmin, xmax)
ax.set_ylim(hmin, hmax)
ax.set_xlabel('x (m)')
ax.set_ylabel('h (m)')
ax.plot(x, h_out[0,:], 'b:')
line, = ax.plot([], [], lw=2)
fig.tight_layout()
pl.close()

In [ ]:
# ANIMATION CREATION PHASE 2
# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    line.set_data(x, h_out[i,:])
    return (line,)

# Call the animator. Keyword blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=h_out.shape[0], interval=40, blit=True)

HTML(anim.to_html5_video())

___
### 4. Analysis of the experiment
Below you find the end result of the experiment. All three variables $h^{\prime}$, $u^{\prime}$ and $v^{\prime}$ are plotted.

In [ ]:
pl.figure(figsize=figsize_l)
pl.subplot(121)
pl.plot(x, h , 'b-')
pl.plot(x, h0, 'b:')
pl.xlabel('x (m)')
pl.ylabel('h\' (m)')
pl.subplot(122)
pl.plot(x, u, 'b-', label='u')
pl.plot(x, v, 'r-', label='v')
pl.xlabel('x (m)')
pl.ylabel('u, v (m/s)')
pl.legend(loc=0, frameon=False);
pl.tight_layout()

1. What do you see in the figure above? Is this what you expected? Why does the surface not become flat?
2. Replot the figure above, but normalize $x$ again with $L_r$ and limit the range between -8 and 8. Within how many Rossby radii is the initial perturbation contained?
3. Calculate the potential vorticity at the end of the simulation and compare it to the initial value. How has it changed?
4. Rerun the experiment for an earth that spins four times faster (set `f_multiplier=4.`) and for one that spins four times slower (set `f_multiplier=0.25`).
5. What is the influence of the rotation rate on the phase velocity and group velocity of the waves? Do your observations confirm the theory?
___

### 5. Geostrophic adjustment
With the help of the potential vorticity, it is possible for this experiment to predict the final steady state out of the initial conditions. The formulation of the steady state is this specific experiment is

$$
\begin{eqnarray}
\nonumber
0 & = & f_0 v^\prime - g \dfrac{\partial h^\prime}{\partial x}
\\
\nonumber
0 & = & -f_0 u^\prime
\\
\nonumber
0 & = &
- H \dfrac{\partial u^\prime}{\partial x}.
\end{eqnarray}
$$

We know that throughout the geostrophic adjustment the potential vorticity has not changed, thus

$$
q = q_0
$$
___
1. Why is the process that you have modelled called geostrophic adjustment? TIP: Think about how to define geostrophic wind speed in the shallow water equations.
2. Derive an equation that relates the final height $h^{\prime}$ to the initial potential vorticity $q_0$. This is a differential equation, you do not have to solve it.
___

Below you find a solver for the final state that takes the coriolis parameter as an input.

In [ ]:
q0 = (np.gradient(v0,dx) + f0) / (h0 + H)

def calc_steady_state():
    c1 = g/(f0*dx**2)

    # Matrix solver
    M = np.zeros((nx, nx))

    for i in range(1,nx-1):
        M[i,i-1] = c1
        M[i,i  ] = -(2.*c1 + q0[i])
        M[i,i+1] = c1

    M[ 0, 0] = 1.
    M[-1,-1] = 1.

    h_mat = -1.*np.ones(nx)*f0
    h_mat[ 0] = H
    h_mat[-1] = H

    h = np.linalg.inv(M).dot(h_mat)
    v = g/f0*np.gradient(h, dx)
    
    return h, v

h_inv, v_inv = calc_steady_state()

In [ ]:
pl.figure(figsize=figsize_l)
pl.subplot(121)
# ADD HERE THE SOLUTION FOR H
pl.plot(x, h, 'k--', label='model')
pl.xlabel('x (m)')
pl.ylabel('h\' (m)')
pl.legend(loc=0, frameon=False)
pl.subplot(122)
# ADD HERE THE SOLUTION FOR V
pl.plot(x, v, 'k--', label='model')
pl.xlabel('x (m)')
pl.ylabel('v (m/s)')
pl.legend(loc=0, frameon=False);
pl.tight_layout()

1. Add the solution acquired from the matrix inversion to the plot above, and normalize $x$ with the Rossby radius. Is the result the same? If not, explain the differences.
2. Explain the pattern in the final solution for $v$. What are the similarities between this pattern and a zonal cross section through the center of a low pressure system in the lower troposphere? 

___
End of tutorial.
___